# Notes
This code-demo was digitized from the "H2_4_Dautel_Fenton_Sosna_AAR_Demo_wCode_v4.pdf" file attached.
All credit goes to FERC for providing this code.

WARNING: No warranty is provided, and none of the code included in this package should be used without first an engineering-review and code-quality assessment. Code is provided as-is for reference purposes only.



# FERC-NOAA Hourly AAR Demonstration
Originally coded for the 2022 FERC Technical Conference: Increasing Market and Planning Efficiency through Improved Software (Docket No. AD10-12-013)

Contents: 
Steps in the Setup or Maintenance Timeframe
Steps in Real Time to Calculate and Archive AARs
Compute forecast margins and adjusted temperature forecasts
Compute AARs for all the Candidate Rating Points
Detour: AAR charts showing temp and solar intensity
Apply the most limiting AAR line rating and update the archive

In [ ]:
# Load relevant Python libraries
import numpy as np
import os
import pandas as pd
import shutil
from datetime import datetime, timedelta
from matplotlib import pyplot as plt
from matplotlib import cm
import seaborn as sns

# Load custom libraries
## This loads the NOAA NBM data and has a pygrib dependency that only works on OSX and Linux 
import nbm as n 

## This contains functions for calculating line ratings based on the IEEE 738-2012 standard
import LineRatings as lr

# Set plot params
plt.rcParams["figure.figsize"] = (12,8)
plt.rcParams.update({'font.size': 18})

# Steps in the Setup or Maintenance Timeframe
## For branches rated with AARs, load their data



In [ ]:
# Load Branch_AAR_data, which mostly comes from the RTS-GMLC system, plus some technical data for what
#   seem like the corresponding ACSR conductors
df_BranchAAR = pd.read_csv('Branch_AAR_data.csv')

# Also store as an array, for the more computationally intensive steps
ar_BranchAAR = np.array(df_BranchAAR) 
df_BranchAAR.head(10)

## Load geographic/local data for all candidate rating points along those branches


In [ ]:
# Load Candidate_rating_point_data, these are the points on every line segment, that we will match with 
# NBM forecasts to calculate ratings
df_RatingPoints = pd.read_csv('Rating_Points_data.csv')

# Also store as an array, to be used for the more computationally intensive steps
ar_RatingPoints = np.array(df_RatingPoints) 
df_RatingPoints.head(10)

## Repackage data into exact format to be input into rating calculation function
This takes a few minutes to run, but only needs to be run when you set up your AAR program or you change any of your line, geographic, or local data changes.

In [ ]:
calc_arguments = lr.repackage_args(df_BranchAAR, df_RatingPoints)

To save time during the demo, instead of running the above command to repackage the system data, we will instead load a numpy array that has already been repackaged.

In [ ]:
# Load or save calc_arguments

# To save data as new fixed .npy file:
#np.save('calc_arguments.npy',calc_arguments)

#calc_arguments_temp = np.load('calc_arguments.npy', mmap_mode='r') # to load data from .npy
#calc_arguments = np.copy(calc_arguments_temp) # copy to new array, so not in memory map mode

## reate line ratings archive
Here we create a Pandas dataframe that we will append new records to. In practice would likely be SQL or Oracle or such.

In [ ]:
# Create empty dataframe with column names
Ratings_archive = pd.DataFrame(columns = ['AAR_cycle_Year_UTC','AAR_cycle_Month_UTC','AAR_cycle_Day_UTC',
'AAR_cycle_Hour_UTC','line','type','fhour','rating_Amps']).astype(int)

# View empty dataframe
Ratings_archive.head()

## Steps in Real Time to Calculate and Archive AARs
Now that the AAR program has been set up, we are ready at any time to load the most recent data from the NOAA National Blend of Models (NBM) forecast, and calculate a fresh set of AARs.
### Load relevant National Blend of Models (NBM) forecast data from the NOAA/NOMADS servers
- Files are downloaded individually from NOMADS server by calling the Temp2m() function from the NBM library imported above, which requests and downloads each file. 
- Saved files are gridded temperature forecast data, stored in grib2 format. 
- The NBM files are typically published 1 hour 10 mins after the initiation hour. Since our example code (loaded in the next step) was downloaded at 35 mins past the hour, we were able to successfully download the data published 1 hour 35 mins previously (using a previousHours = 1 parameter to our call to Temp2m() ). In some cases, you will have to use previousHours = 2. In production, transmission providers may need to test to see what the most recent available suite of forecasts is. 
- The Temp2m() function defaults to previousHours = 1, if no value is passed when the function is called. But we pass this argument explicitly below for clarity. 
- The Temp2m() (as written) also defaults to downloading data within the box bounded by -120 longitude on the left, -110 longitude on the right, 38 latitude on the top, and 31 latitude on the bottom, in order to capture the locations of the rating points in our test system. These defaults can be changed to reflect any other locations, or different values can be passed explicitly as arguments to Temp2m(). 
- The Temp2m() function also defaults to downloading all available grib files from hours 1 to 268. You can set a smaller set of files to download by specifying the forecast hours (as a list or range) in the argument forecastHourSet. Downloading a full suite of grib2 files for hours 1 through 268 can take several minutes. To demonstrate this more quickly, we'll download just for hours 1 to 70 (and then use a full set of forecasts previously downloaded).

In [ ]:
# Prior to running this cell, manually create a folder 'NBM/' as a subfolder to your working directory
#  if one doesn't already exist.

# For the demo, we'll show downloading only grib2 files for the first 60 forecast hours.
#initDateTime, initHour, successHours = n.Temp2m(previousHours=1,forecastHourSet=range(1,60))

# Below is the "normal" code to download the full set of grib2 files.
initDateTime, initHour, successHours = n.Temp2m(previousHours=1,forecastHourSet = range(1,268))

## Adjustment for presentation, loading previous forecasts
Since we'll use previously downloaded data, we'll now delete the files we just downloaded and copy the older files to that directory.

In [ ]:
# Clear files from saveDirectory
#for files in os.listdir('NBM/'):
#    path = os.path.join('NBM/', files)
#    try:
#        shutil.rmtree(path)
#    except OSError: #if rmtree() command doesn't work on local OS
# os.remove(path)
#os.rmdir('NBM/')

# Copy previously downloaded files to NBM directory
#shutil.copytree('NBM_demo', 'NBM/');

In [ ]:
# Create the variables that would have been created if we had run Temp2m() instead of just uploading our data.
# These are needed in the line rating calculations below.

#initDateTime = datetime(2022, 6, 19, 14, 25, 26, 431672) # UTC date/time at which we downloaded our data
#initHour = 14
#successHours = list(range(1,38)) + list(range(40,191,3)) + list(range(196,263,6))

## Process downloaded grib2 files
Process downloaded grib2 files by:
- Extracting grib2 data into numpy arrays 
- For each forecast hour (1 to 240), associating each candidate rating point with its nearest forecast point. 
- Compute forecast margin and apply to forecast. Not wanting to make any suggestions about what an appropriate forecast margin is, we implement a forecast margin that is a simple multiple of the standard deviation (where our multiple was selected at random from a range of candidates). 
- A note: the standard deviations in the files do not represent forecast accuracy relative to observed data but instead repersent how closely the different models in the NBM agree with each other on a given forecast.

In [ ]:
# Convert NBM i,j points into array that can be passed to n.processNBMGribFiles()
ar_RatingPointsIJs = np.array(df_RatingPoints[['NBM_i', 'NBM_j']])

temp, tempSTD = n.process_nbm_grib_files(initHour, successHours, ar_RatingPointsIJs)

# Display first row of temp, showing all 240 forecasted temps for that rating point
temp[0],temp.shape

## Compute forecast margins and adjusted temperature forecasts

How to calculate forecast margins and what confidence levels to reflect are an area for more research and discussion. For purposes of having numbers to plot, we implemented an approach, but we do not believe our approach is methodolgically valid. In general, our method reflects published annual average mean absolute errors for temperature forecasts over similar timeframes, and then scales those values up or down depending on the reported values for forecast standard deviation (which is a measure of how well the underlying NBM models agree, not a proxy for standard error of the forecast itself). That yields an estimated standard error, which we then scale up by 3.4 (a randomly chosen multiple) in order to achieve some unstated confidence level.

This relies on an assumption that the forecast errors are normally distributed. However, we know from weather experts and published annual average mean absolute errors for temperature forecasts that this is an erroneous assumption.

How to more accurately estimate forecast margins is an important consideration for implementing AARs and should be included in further research/discussion.

Below, we calculate estimated standard error, using a blackbox function:

In [ ]:
"""
This function was created only for the purposes of being able to compute values for demonstration,
but this function is not methodologcially valid, and should not be used for any purposes other
than demonstration.
"""
def estForecastStdErr(STD,fhour):

    # Compute an estimate for standard error based purely on forecast hour
    #   (this is designed to roughly match data published for a different but somewhat similar forecast)
    stdErr = fhour * 0.035 + 0.5

    # Scale the stdErr up or down depending on how well the models underlying the NBM forecast
    #   agree with each other (as measured by the tempSTD returned from the NBM data).
    scale = STD*0.06 + 0.3

    return stdErr*scale

In [ ]:
# Here we apply the estimated forecast STD error function, again for demonstration purposes only
estStdErr = np.empty(shape=tempSTD.shape) # create empty array for estimated standard error of forecast
estStdErr[:] = np.nan

# Compute estimated standard error using the estForecastStdErr() function
for fhour in range(0,tempSTD.shape[1]): 
    estStdErr[:,fhour] = estForecastStdErr(tempSTD[:,fhour],fhour)

estStdErr

In [ ]:
forecastMargin = np.empty(shape=tempSTD.shape) # create empty array for forecast margin
forecastMargin[:] = np.nan

forecastMargin = estStdErr*3.4
# Show forecastMargin and shape
(forecastMargin, forecastMargin.shape)

In [ ]:
# Select 35 series (every 100 of the 3513 rows) in forecastMargin to plot as scatter vs forecast hour.
for row_num in range(0,forecastMargin.shape[0],100):
    x = range(forecastMargin.shape[1]) # create x values as the forecast hours
    y = forecastMargin[row_num,:]*9/5 # create y values as forecast margins converted from C intervals to F intervals
    sns.scatterplot(x = x,y = y,size=1,legend=False)

plt.xlabel("forecast hour"); plt.ylabel("forecast margin (F)"); plt.gca().set_ylim(bottom=0)
plt.title('Forecast margin vs forecast hour for 35 candidate rating points')
plt.show()

In [ ]:
# Compute the adjusted temperature (which incorporates the forecast margin) 
# This will be used to calculate the AARs
adjustedTemp = temp + forecastMargin

# Show values and shape
(adjustedTemp, adjustedTemp.shape)

Below we examine the different temperature plots at a randomly selected candidate rating point, across the 240 forecast hours.

For reference, we also show a straight line for T = 105F, which is a common temperature assumption for static ratings or for summer seasonal ratings.

Note the impact of the forecast margin on the adjusted temperatures, which are especially high and diverge from the expected temperatures toward the end of the timeframe. AARs based on these adjusted temperatures would lead to less transfer capacity on the hottest days in this period. If the forecast margin were accurate these would be the right ratings to use, however, if the forecast margin relies of false assumptions like normally distributed temperature errors, then this would be an arbitrary loss of transfer capacity.

In [ ]:
# Plot forecast margin, expected temp, adjusted temp, and T = 105F(a typical summer assumption)
plt.plot(temp[0]*9/5+32) #the unadjusted temp forecast (converted to F)
plt.text(200,75,'expected temp', color = 'xkcd:cerulean')

plt.plot(forecastMargin[0]*9/5) # forecast margin (converted to an F interval)
plt.text(50,23,'forcast margin', color = 'xkcd:emerald green')

plt.plot(adjustedTemp[0]*9/5+32) # adjusted temp (converted to F)
plt.text(120,120,'adjusted temp', color = 'xkcd:scarlet')

plt.plot([0,260],[105,105], linestyle= 'dashed') # for reference, show a typical summer seasonal rating
plt.xlabel("forecast hour"); plt.ylabel("temp (F)")

plt.title('Temp adjustment for selected candidate rating point')
plt.show()

## Compute AARs for all the Candidate Rating Points
### Prepare arguments for calculation

Append the downloaded weather data into the prevously prepared arguments.

The user (or the automated system, in practice) will need to choose how many forecast hours to calculate line ratings for (and therefore which columns in the adjustedTemp array to include as arguments for the calculation). In our case we have set up the calc_arguments array such that it will only hold 240 hours.

In this step, we also exclude column 0 from the adjustedTemp array, which means we are excluding forecast hour f001. We do this because f001 is already in the past by the time we run our calculation.

Thus, for the purposes of the line rating calculation, what was f002 becomes our first forecast hour.

In [ ]:
# Take only the 240 temperature forecasts after hour f001.
lr.update_args(calc_arguments, adjustedTemp[:,1:241], initDateTime)

## Execute the calculation of ratings at each candidate rating point
Each argument for the calculate_thermal_rating() function is passed as a column of the calc_arguments array.

We write the output of the calculate_thermal_rating() function to the last column of calc_arguments (which will be further used as an argument further below for determining the overal ratings for each line).

In [ ]:
calc_arguments[:,-1] = lr.calculate_thermal_rating(
T_a = calc_arguments[:,4],
T_s = calc_arguments[:,5],
D = calc_arguments[:,6],
emis = calc_arguments[:,7],
absorp = calc_arguments[:,8],
R = calc_arguments[:,9],
W_v = calc_arguments[:,10],
W_a = calc_arguments[:,11],
elev = calc_arguments[:,12],
air_qual = calc_arguments[:,13],
year = calc_arguments[:,14],
day_of_year = calc_arguments[:,15],
start_hour = calc_arguments[:,16],
latitude = calc_arguments[:,17],
longitude = calc_arguments[:,18],
time_zone = calc_arguments[:,19],
line_azimuth = calc_arguments[:,20],
verbose=0, 
)

## Detour: AAR charts showing temp and solar intensity
### Show calculated candidate ratings along one line during first forecast hour
Here you see the AAR ratings at all the candidate rating points for a given rating line and forecast hour. Only the lowest rating will be applied to the entire line and archived. The relationship between normal and emergency rating types is as expected.

In [ ]:
EXAMPLE_LINE = 65
EXAMPLE_HOUR = 1

# Create dataframe with columns for line (0), point (1), type (2), forecast hour (3), and candidate rating (21).
df = pd.DataFrame(calc_arguments[:,[0,1,2,3,21]],
columns = ['line','point','type','fhour','rating']
)
# masks to filter data
mask = (df['line'] == EXAMPLE_LINE) & (df['fhour'] == EXAMPLE_HOUR)
# Reshape df to facilitate plotting
df_reshaped = df[mask].pivot(index='point', columns='type', values='rating')

df_reshaped.plot(title='Candidate Ratings for line 65, forecast f001')
plt.legend(['Normal', 'Long-term emergency', 'Short-term emergency'])
plt.ylabel("Rating (Amps)")
plt.xlabel('Candidate rating point')
plt.show()

## Show calculated candidate normal ratings for one line from early morning to noon
Note how the AAR ratings decline from 5am to 2pm for a given rating line and forecast hour, which is as expected as temperatures rise and solar heating intensifies.

In [ ]:
EXAMPLE_LINE = 65
EXAMPLE_TYPE = 0 #0 for normal ratings

# Define new column which will have local time
#  (valid for our fhour range of 16 to 40)
df['local_time'] = df['fhour']-16

# Create masks to filter data
mask = (((df['line'] == EXAMPLE_LINE) & (df['type'] == EXAMPLE_TYPE)) & (df['local_time'] <= 14)) & (df['local_time'] >= 5)

# Reshape df to facilitate plotting
df_reshaped = df[mask][['point','local_time','rating']].pivot(index='point', 
columns='local_time', 
values='rating')
colors = cm.cool(np.linspace(0, 1, 31-21))
df_reshaped.plot(title='Candidate normal ratings for line 65\nfrom 5am to 2pm local time',color = colors)

plt.ylabel('Rating (Amps)')
plt.xlabel('Candidate rating point')                                       # places legend on side
plt.legend(['5am','6am','7am','8am','9am','10am','11am','noon','1pm','2pm'],bbox_to_anchor = (1, 1)) 

plt.show()

## Show calculated rating for one candidate rating point across a 24-hour cycle, along with ambient temp
As expected, there is an inverse relationship between the temperature and the AAR rating for a given candidate rating point.

In [ ]:
EXAMPLE_LINE = 65

# Get new dataframe (like the previous) that adds a column for T_a (air temp) (column 4)
df = pd.DataFrame(calc_arguments[:,[0,1,2,3,4,21]],
columns = ['line','point','type','fhour','T_a','rating']
)

# Define new column which will have local time
#  (valid for our fhour range of 16 to 40)
df['local_time'] = df['fhour']-16

# Create masks to filter the data
mask1 = (df['line'] == EXAMPLE_LINE)
mask2 = (df['local_time'] <= 24) & (df['local_time'] >= 0)
mask3 = df['point'] == 0
mask4 = df['type'] == 0 #normal
mask = ((mask1 & mask2) & mask3) & mask4

# Plot rating and T_a vs local time
ax = df[mask].plot(x='local_time', y = ['rating','T_a'], secondary_y = 'T_a')

ax.set_ylabel('Rating (Amps)')
ax.set_title('Rating and air temp across a day')
ax.right_ax.set_ylabel('Air Temp (C)')

plt.show()

It is impossible to determine from that chart how much of the rating change is due to ambient temperature change (which is clearly correlated) and how much is due to changes in solar heating across the day.

So below we plot rating for the same candidate rating point with the ambient air temperature and all other conditions constant at local hour 10, except that we let solar heating vary across the 24
hours of a day.

In [ ]:
# Create masks to filer the data
mask1 = df['line'] == 65
mask2 = df['fhour']== 10+16
mask3 = df['point'] == 0
mask4 = df['type'] == 0 #normal
mask = ((mask1 & mask2) & mask3) & mask4

x = calc_arguments[mask] # filter data

# Calculate line ratings for the filtered data point, except for start_hour, which can vary from 0 to 23
y = lr.calculate_thermal_rating(
T_a = x[:,4],
T_s = x[:,5],
D = x[:,6],
emis = x[:,7],
absorp = x[:,8],
R = x[:,9],
W_v = x[:,10],
W_a = x[:,11],
elev = x[:,12],
air_qual = x[:,13],
year = x[:,14],
day_of_year = x[:,15],
start_hour = np.array(range(24)),
latitude = x[:,17],
longitude = x[:,18],
time_zone = x[:,19],
line_azimuth = x[:,20],
verbose=0, 
)

# Fill area
new_x = range(4, 20)
new_y_low = 1130
new_y_high = y[4:20]
plt.fill_between(new_x, new_y_low, new_y_high , color = 'yellow', alpha=0.2)

plt.ylim([1100,1280])
plt.plot(y, 'o-')
xvals = [0,4,4,19,19,24]
yvals = [y[0], y[0], y[12], y[12], y[0], y[0]]
plt.plot(xvals, yvals)
plt.ylabel('Rating (Amps)')
plt.xlabel('Hour of Day')
plt.title('Rating across day (holding air temperature constant)')

plt.show()

End of detour

## Apply the most limiting AAR line rating and update the archive
For each line, compute its line rating for a given rating type and forecast hour as the minimum of all of the candidate line ratings for that rating type and forecast hour.

In [ ]:
# Determine the minimum rating per line, per type, per forecast hour
unique_ratings_per_AAR_cycle = df.groupby(['line', 'type','fhour'])['rating'].min().reset_index().astype(int)

unique_ratings_per_AAR_cycle = unique_ratings_per_AAR_cycle.rename(columns = {'rating':'rating_Amps'}) 

# Add the AAR cycle date/time info
## Based on having started the AAR calculation cycle and requested a NBM forecast 1 hour prior to the
#   request hour.  Would need to adjust the below line if ever used a different amount of time (say 2 hours,
#   maybe if the data for 1 hour previously wasn't available due to a server error) prior to request hour.
AAR_cycle_UTC = initDateTime + timedelta(hours=1) 

# Record the line ratings and other metadata in a small array
unique_ratings_per_AAR_cycle = unique_ratings_per_AAR_cycle.join(pd.DataFrame(
    {'AAR_cycle_Year_UTC': AAR_cycle_UTC.strftime('%Y'),
    'AAR_cycle_Month_UTC': AAR_cycle_UTC.strftime('%m'),
    'AAR_cycle_Day_UTC': AAR_cycle_UTC.strftime('%d'),
    'AAR_cycle_Hour_UTC': AAR_cycle_UTC.strftime('%H')},
    index=df.index))

unique_ratings_per_AAR_cycle.head()  # view values

Up to this point in our demo, ratings have been calculated for the specific instants at the top of each hour that the NBM forecasts apply to. To produce a useable and reliable line rating across each hourly period, we set each such period rating equal to the lesser of the instant ratings calculated for the start and end if each hourly period.

In [ ]:
# Compare the rating at the top of hour t with the rating at the top of hour t+1
## and take the lowest, which will become the rating column
updated_rating_Amps = []
i = 0

for i in range(0, len(unique_ratings_per_AAR_cycle['rating_Amps'])):
    updated_rating_Amps.append(min(unique_ratings_per_AAR_cycle['rating_Amps'][i:(i+2)]))

In [ ]:
# Look at the share of hourly ratings we update in the conservative column
count_dif = sum(map(lambda x,y: bool(x-y),unique_ratings_per_AAR_cycle['rating_Amps'],updated_rating_Amps))

# 39% of them changed
(count_dif/len(unique_ratings_per_AAR_cycle['rating_Amps']))

In [ ]:
# Add the updated ratings to the hourly rating dataframe
unique_ratings_per_AAR_cycle.insert(4,'updated_rating_Amps',updated_rating_Amps)

# And look at them, columns 4 and 5:
unique_ratings_per_AAR_cycle.head()

In [ ]:
# Looking at the first 24h for the top of the hour and conservative ratings
plt.plot(unique_ratings_per_AAR_cycle['rating_Amps'].loc[(
                                            unique_ratings_per_AAR_cycle['line']==0)& 
                                            (unique_ratings_per_AAR_cycle['type']==0)][0:24]) 

plt.plot(unique_ratings_per_AAR_cycle['updated_rating_Amps'].loc[(
                                            unique_ratings_per_AAR_cycle['line']==0)& 
                                            (unique_ratings_per_AAR_cycle['type']==0)][0:24], color = 'xkcd:tangerine') 

plt.legend(['rating_Amps', 'updated_rating_Amps']) 
plt.xlabel('Hour')
plt.ylabel('Rating (Amps)')
plt.title('Top-of-the-hour vs hourly duration ratings')

plt.show()

In [ ]:
# Now we will replace the updated ratings (based on the lower of the t or t+1 ratings) 
## with the original top of the hour ratings before we use them and update the archive
unique_ratings_per_AAR_cycle['rating_Amps'] = unique_ratings_per_AAR_cycle['updated_rating_Amps']
unique_ratings_per_AAR_cycle.drop('updated_rating_Amps', axis = 1, inplace = True)
unique_ratings_per_AAR_cycle.head()

In [ ]:
# Update the archive
Ratings_archive = pd.concat([Ratings_archive, unique_ratings_per_AAR_cycle],ignore_index = True)

Ratings_archive.head() # View values